In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/commonlit-evaluate-student-summaries/sample_submission.csv
/kaggle/input/commonlit-evaluate-student-summaries/prompts_train.csv
/kaggle/input/commonlit-evaluate-student-summaries/summaries_test.csv
/kaggle/input/commonlit-evaluate-student-summaries/summaries_train.csv
/kaggle/input/commonlit-evaluate-student-summaries/prompts_test.csv


In [2]:
def print_tuple(obs):
    print("\033[1mprompt_question: \033[0m", obs.prompt_question)
    print()
    print("\033[1mprompt_title: \033[0m", obs.prompt_title)
    print()
    print("\033[1mprompt_text: \033[0m")
    print(obs.prompt_text)
    print()
    print("\033[1msummary: \033[0m")
    print(obs.text)
    print()
    print("\033[1mWording: \033[0m")
    print(obs.wording)
    print()
    print("\033[1mContent: \033[0m")
    print(obs.content)
    

In [3]:
#Filepath
FILE_PATH='/kaggle/input/commonlit-evaluate-student-summaries'
#train set
pmt_train=pd.read_csv(os.path.join(FILE_PATH,'prompts_train.csv'))
sum_train=pd.read_csv(os.path.join(FILE_PATH,'summaries_train.csv'))
df_train=pmt_train.merge(sum_train,how='inner',on='prompt_id')

In [4]:
#df_train
tuple1=df_train.iloc[0]
print_tuple(tuple1)

prompt_question:  Summarize at least 3 elements of an ideal tragedy, as described by Aristotle.

prompt_title:  On Tragedy

prompt_text: 
Chapter 13 
As the sequel to what has already been said, we must proceed to consider what the poet should aim at, and what he should avoid, in constructing his plots; and by what means the specific effect of Tragedy will be produced. 
A perfect tragedy should, as we have seen, be arranged not on the simple but on the complex plan. It should, moreover, imitate actions which excite pity and fear, this being the distinctive mark of tragic imitation. It follows plainly, in the first place, that the change of fortune presented must not be the spectacle of a virtuous man brought from prosperity to adversity: for this moves neither pity nor fear; it merely shocks us. Nor, again, that of a bad man passing from adversity to prosperity: for nothing can be more alien to the spirit of Tragedy; it possesses no single tragic quality; it neither satisfies the moral

In [5]:
#we actually dont want student_id or prompt_id for training hence dropping those.
df_train.drop(['student_id','prompt_id'],axis=1,inplace=True)

### Tokenizing and preprocessing

We will be using tokenizers like Bert and RoBERTa

In [6]:
import numpy as np
from string import Template
import tqdm
import torch

from sklearn.decomposition import PCA

from transformers import (BertTokenizer, BertModel , 
                         T5Tokenizer, T5ForConditionalGeneration , 
                         AutoModelForCausalLM, BitsAndBytesConfig, AutoTokenizer , 
                         AutoModel , DebertaModel , ElectraModel , 
                         RobertaModel , RobertaTokenizer)

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l